A notebook to make p-value images from the results of whole_bain_linear_mdl_fit

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import os
from pathlib import Path
import pickle

import imageio
import numpy as np
import pandas as pd
import pyqtgraph as pg

from janelia_core.stats.regression import grouped_linear_regression_boot_strap_stats
from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.utils.data_saving import append_ts

In [3]:
%matplotlib qt

## Parameters go here

In [4]:
ps = {}

# Location of results of whole_brain_linear_mdl_fit
ps['results_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['results_file'] = 'whole_brain_boot_strap_2019_10_29_10_19_57_489328.pkl'

# The alpha level for forming confidence intervals (we don't look at confidence intervals but still compute them)
ps['alpha'] = .01

# Specify where the original datasets are located - we use these for determining the position of the rois
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'
ps['dataset_folder'] = 'extracted'
ps['dataset_base_folder'] = r'K:\\SV4'
ps['roi_group'] = 'rois_5_25_25'

# Specify where we save images
ps['save_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'whole_brain_stats_images'

## Load the results

In [5]:
rs_file = Path(ps['results_folder']) / ps['results_file']
with open(rs_file, 'rb') as f:
    rs = pickle.load(f)
    
one_hot_vars = rs['one_hot_vars']
before_bs_rs = rs['before_bs_rs']
after_bs_rs = rs['after_bs_rs']

## Load a dataset

Because the rois are in the same location for each dataset, we can just look at the first dataset to find the position of the rois

In [6]:
# Read in dataset locations
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

# Read in the first dataset
dataset_path = (Path(ps['dataset_base_folder']) / data_locs['Main folder'][0] / data_locs['Subfolder'][0] / 
                    Path(ps['dataset_folder']) / '*.pkl')
dataset_file = glob.glob(str(dataset_path))[0]

with open(dataset_file, 'rb') as f:
    dataset = ROIDataset.from_dict(pickle.load(f))

In [28]:
dataset_file

'K:\\SV4\\CW_17-08-23\\L1-561nm-ROIMonitoring_20170823_145226.corrected\\extracted\\CW_17-08-23-L1_dataset.pkl'

## Get ROI locations for first dataset

In [7]:
rois = dataset.roi_groups[ps['roi_group']]['rois']
if len(rois) != len(before_bs_rs):
    raise(RuntimeError('Number of rois in dataset does not match number of rois statistics are calculated for.'))
n_rois = len(rois)

## Calculate statistics

In [8]:
before_stats = [grouped_linear_regression_boot_strap_stats(rs[0], ps['alpha']) for rs in before_bs_rs]
after_stats = [grouped_linear_regression_boot_strap_stats(rs[0], ps['alpha']) for rs in before_bs_rs]

## Load mean image

In [9]:
mn_img = dataset.stats['mean']

## Generate and save images

In [23]:
# Make folder to save results into
image_folder = append_ts(ps['save_str'])
save_folder_path= Path(ps['save_folder']) / image_folder
os.makedirs(save_folder_path)

In [24]:
# Save the mean image
mn_image_path = save_folder_path / 'mean.tiff'
imageio.mimwrite(mn_image_path, mn_img)

In [25]:
# Generate p-value images
im_shape = mn_img.shape

n_vars = len(one_hot_vars)

before_p_images = [None]*n_vars
after_p_images = [None]*n_vars

for v_i in range(n_vars):
    after_im = np.zeros(im_shape, dtype=np.float32)
    before_im = np.zeros(im_shape, dtype=np.float32)
    cur_var = one_hot_vars[v_i]
    for r_i in range(n_rois):
        cur_voxel_inds = rois[r_i].voxel_inds
        after_im[cur_voxel_inds] = -1*np.log(after_stats[r_i]['non_zero_p'][v_i])
        before_im[cur_voxel_inds] = -1*np.log(before_stats[r_i]['non_zero_p'][v_i])
        
    after_p_images[v_i] = after_im 
    before_p_images[v_i] = before_im

In [26]:
# Save p-value images

after_folder_path = save_folder_path / 'after'
before_folder_path = save_folder_path / 'before'

os.makedirs(after_folder_path)
os.makedirs(before_folder_path)

for v_i in range(n_vars):
    after_im_name = one_hot_vars[v_i] + '_after.tiff'
    before_im_name = one_hot_vars[v_i] + '_before.tiff'
    
    after_im_path = after_folder_path/after_im_name
    before_im_path = before_folder_path/before_im_name
    
    imageio.mimwrite(after_im_path, after_p_images[v_i])
    imageio.mimwrite(before_im_path, before_p_images[v_i])

In [27]:
# Save parameters
param_path = save_folder_path / 'params.pkl'
with open(param_path, 'wb') as f:
    pickle.dump(ps, f)

In [30]:
one_hot_vars

['beh_before_Q',
 'beh_before_F',
 'beh_before_B',
 'beh_after_Q',
 'beh_after_F',
 'beh_after_B',
 'beh_interact_QQ',
 'beh_interact_QF',
 'beh_interact_QB',
 'beh_interact_FQ',
 'beh_interact_FF',
 'beh_interact_FB',
 'beh_interact_BQ',
 'beh_interact_BF',
 'beh_interact_BB',
 'subject_CW_17-08-23-L2',
 'subject_CW_17-08-23-L4',
 'subject_CW_17-08-24-L4',
 'subject_CW_17-08-24-L5',
 'subject_CW_17-08-26-L1',
 'subject_CW_17-08-26-L2',
 'subject_CW_17-08-26-L4',
 'subject_CW_17-08-26-L5',
 'subject_CW_17-08-26-L6',
 'subject_CW_17-08-27-L5',
 'subject_CW_17-08-28-L1',
 'subject_CW_17-08-28-L2',
 'subject_CW_17-08-29-L2',
 'subject_CW_17-09-01-L1',
 'subject_CW_17-09-01-L3',
 'subject_CW_17-11-02-L3',
 'subject_CW_17-11-03-L1',
 'subject_CW_17-11-03-L3',
 'subject_CW_17-11-03-L6',
 'subject_CW_17-11-03-L7',
 'subject_CW_17-11-04-L1',
 'subject_CW_17-11-04-L3',
 'subject_CW_17-11-04-L4',
 'subject_CW_17-11-05-L6',
 'subject_CW_17-11-06-L3',
 'subject_CW_17-11-07-L3',
 'subject_CW_17-11-

In [31]:
62-15

47

In [32]:
-np.log(.002)

6.214608098422191